In [ ]:
import numpy as np
import math

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [ ]:
def householder(vec):
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)

    v_norm = np.linalg.norm(vec, ord=2) #norm of the vector, corrected for order

    u = vec.copy() #operational vector

    u[0] = (vec[0]**2 - v_norm**2) / (vec[0] + v_norm)
    u_norm = np.linalg.norm(u, ord=2) 
    u = (u / u_norm)[np.newaxis]
    
    N = len(vec) #Vector size/length/shape
    H = np.eye(N) - 2 * np.matmul(u.T, u)
    V1 = np.dot(H, vec)
    

    return V1 , H
    

Test your function using tests below:

In [ ]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
#print(v)
v1, h = householder(v)


assert_allclose(np.dot(h, v1), v, atol=1e-8)
print(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1, atol=1e-8)
print(np.dot(h, v), v1)

[1. 2. 3.] [1 2 3]
[3.74165739e+00 2.22044605e-16 0.00000000e+00] [3.74165739e+00 2.22044605e-16 0.00000000e+00]


In [ ]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [ ]:
def qr_decomp(a):
    
    a1 = np.array(a, copy=True, dtype=float)
    m, n = a1.shape
    Q, R = np.eye(m), a1
    
    for i in range(n): 
        _, H = householder(R[i:, i]) #making use of the function just defined
        h_i = np.eye(m) #Identity
        h_i[i:, i:] = H
        
        Q = np.matmul(Q, h_i)
        R = np.matmul(h_i, R)
    
    return Q, R

In [ ]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [ ]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(5, 3))
q, r = qr_decomp(a)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(5), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [ ]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

In [ ]:
print('Q', q)
print('----------')
print('R', r)
print('----------')
print('QQ', qq)
print('----------')
print('RR', rr)
print('----------')

Q [[ 0.13665049  0.53601299 -0.09369752  0.7697136   0.30459557]
 [ 0.56035895  0.0935397  -0.53326881  0.01839528 -0.62652547]
 [ 0.19725922  0.65948912  0.60068463 -0.32384673 -0.24589462]
 [ 0.62498418 -0.50418303  0.52144688  0.28453698  0.04822969]
 [ 0.48765568  0.12171264 -0.27224305 -0.47049398  0.67223293]]
----------
R [[ 1.40152769  1.2514379   0.89523615]
 [ 0.          0.84158252  0.68447942]
 [-0.         -0.          0.5496046 ]
 [ 0.          0.         -0.        ]
 [ 0.         -0.          0.        ]]
----------
QQ [[-0.13665049  0.53601299  0.09369752  0.661619   -0.49749149]
 [-0.56035895  0.0935397   0.53326881 -0.52477245 -0.34276292]
 [-0.19725922  0.65948912 -0.60068463 -0.37879015  0.14784752]
 [-0.62498418 -0.50418303 -0.52144688  0.18967657 -0.21750907]
 [-0.48765568  0.12171264  0.27224305  0.32774225  0.75222783]]
----------
RR [[-1.40152769 -1.2514379  -0.89523615]
 [ 0.          0.84158252  0.68447942]
 [ 0.          0.         -0.5496046 ]
 [ 0.       

*Enter your explanation here* (10% of the total grade, peer-graded)

QR matrix decomposition has some particularities, for example that Negating Q and R, yields a new, ans still valid QR decomposition. It also turns out that we can arbitrarily flip the signs of (one or more of) the elements on the diagonal of R, that and the rows and columns number are not equal (not a square matrix). So I can in fact imply that those last columns can be whatever I want, and still get a QR decomposed matrix.

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [ ]:
#Class needed for extended procedures

class QR: 
    
    def __init__(self, A): #defines a matrix, changes the shape so no H needed
        m, n = A.shape
        self.R = np.array(A, copy=True, dtype=float)
        self.Q = []
    
        for i in range(n):
            u = np.concatenate((np.zeros(i), self.household(self.R[i:,i])))
            
            self.Q.append(u)
            self.R -= 2 * np.matmul(u.reshape(-1, 1), np.matmul(u.reshape(1, -1), self.R)) 
        
        
    def household(self, vector): #applies householder, does not return H
        vector = np.asarray(vector, dtype=float)
        if vector.ndim != 1:
            raise ValueError("vector.ndim = %s, expected 1" % vector.ndim)

        vector_norm = np.linalg.norm(vector, ord=2)
        u = vector.copy()
        u[0] = (vector[0]**2 - vector_norm**2) / (vector[0] + vector_norm)
        u_norm = np.linalg.norm(u, ord=2)
        u = (u / u_norm)
    
        return u
    
    def Q_trans_multiply(self, X): #Transmutated matrix through multiplication
        
        x = np.array(X, copy=True, dtype=float)
        for u in self.Q[::-1]:
            x -= 2 * np.matmul(u.reshape(-1, 1), np.matmul(u.reshape(1, -1), x))
            
        return x

Now we can test the decomposition ourselves.

In [ ]:
qr = QR(a)
assert_allclose(qr.Q_trans_multiply(qr.R), a)